In [2]:
import data
import numpy as np
import pandas as pd
from main import initiate_models

In [3]:
# Collect data distribution
fname = "../data/Bret_cover_ramets.csv"
df = pd.read_csv(fname,delimiter=',', decimal=',')

# Collect data diameter
fname = "../data/Bret_cover_ramets2.csv"
df2 = pd.read_csv(fname,delimiter=',', decimal='.')

We need to combine the data from diameter and seperate ramets on larger distance. Let's say the area within the diameter circle is 50% filled, and equally distributed. The amount of cloned individuals in this area is $N =\frac{\pi D^2}{8} - 9$, so the amount of individuals I'm expecting at distance $r$ is $\frac{N}{r} = \frac{2}{D}(\frac{\pi D^2}{8} - 9)$

In [9]:
fill = 0.5

# Get mean and std deviation of clones
d_close = df2.ix[:, ['D_3']].values[:,0]
dN_dr = 2/d_close * ((np.pi*d_close**2 / 8) - 9)
print(d_close)
d = df.ix[:, ['NR_3', 'NR_3_distance(cm)']].values
print(d[:,0])
weights, mids = d.T
mean = np.average(mids[~np.isnan(mids)], weights=weights[~np.isnan(weights)])
std = np.average((mids[~np.isnan(mids)] - mean)**2, weights=weights[~np.isnan(weights)])
print("Mean: %.3f, Std: %.3f"%(mean, std))

[ 61.    50.    51.    54.   114.    89.    88.    80.    50.    48.
  41.    27.    34.    66.    77.    93.    98.    37.    54.   103.
  45.    47.   110.    80.    52.    70.    44.    32.    51.    62.
  31.5   44.1   44.    52.5  120.    67.    63.    40.    33.    40.
  34.4   33.82  82.49  62.57  33.    27.39  43.68  68.    40.45  50.53
  40.6   36.   100.    80.    72.    27.54  24.2   61.    57.    20.
  36.    23.    50.    70.    40.    32.   112.    57.    88.    86.17
  30.   170.    56.2   84.16  22.   120.    80.    90.    32.52 101.25
  40.    30.    80.    42.    53.58  35.22  63.2   46.99  42.58 100.
 110.    80.    75.74 100.    45.    30.    40.    40.    40.    56.
  81.4   50.    60.    64.    60.    36.53  56.23  36.19  46.    63.
  40.    36.    57.79  29.42  20.86  41.46  37.59  42.09  96.51  62.
  42.    80.    50.    42.    65.    42.    39.    35.   140.    90.
 110.   110.   110.    80.   150.    90.    60.   140.   130.    50.
 120.    70.    40.   170.  

In [4]:
# Collect modelled patches and corresponding data
models = initiate_models(data.get_params()[0])
data.assign_data(models)
patches = []
for model in models:
    patches += model.patches

In [5]:
labels_data = df.ix[:, ['plot', 'patch']].values
labels = ["%02d%s%s" % (plot, patch[1:], patch[0]) for plot, patch in labels_data]
counts = []
for count, label in zip(weights, labels):
    if not np.isnan(count):
        for patch in patches:
            if patch.id.strip("*") == label and patch.has_data:
                nr = patch.factor * 9
                biom_0 = patch.BR_data[0]
                biom_T = patch.BR_data[2]
                counts.append((biom_0/nr, biom_T/nr, count/nr))
                break
        else:
            raise FileNotFoundError(label)

print(len(counts))

160


We collected the summed probability over the whole period until measurement 3. We now want this final amount of seeds per biomass to the seed probability $p_{seed}$ on every timestep. 

$$ N_{seeds}\big |_{t=T} = \sum_{t=0}^Tp_{seeds}B_t$$

$$= p_{seeds}B_0\sum_{t=0}^Tr^t$$

$$= p_{seeds}B_0\frac{1 - r^{T+1}}{1 - r}$$

$$ p_{seed} = \frac{N_{seeds}}{B_0}\frac{1 - r}{1 - r^{T+1}}$$

Where we can approximate $r$ by

$$B_T = B_0r^T$$
$$r = \left(\frac{B_T}{B_0}\right)^{\frac{1}{T}} $$

In [6]:
T = (data.end_date - data.start_date).days
B_T_B_0 = np.mean([b[1]/b[0] for b in counts])
r_average = B_T_B_0**(1/T)

print(T)
print(B_T_B_0)
print('Average r = %.10f'%r_average)

118
1.6145152003684937
Average r = 1.0040678677


In [18]:
p_seeds = []
N_total = 0
for B_0, B_T, n_total in counts:
    r = (B_T/B_0)**(1/T)
    p = n_total/B_0*(1 - r)/(1 - r**(T+1))
    N_total += n_total
    p_seeds.append(p)
    
print(np.mean(p_seeds), np.std(p_seeds))

0.0030063076304832023 0.0029281088743420293


In [19]:
print("seed_prob = %.3f +/- %.3f"%(np.mean(p_seeds), np.std(p_seeds)))
print("seed_mean = %.3f"%mean)
print("seed_sigma = %.3f"%std)

seed_prob = 0.003 +/- 0.003
seed_mean = 6.701
seed_sigma = 6.756


But now I calculated the expansion over only the counted ramnus clones. These are a tiny fraction of the total expansion of the patch. It makes more sense to look at the expansion of the patch in diameter to estimate $N_{total}$

In [20]:
# Get mean and std deviation of clones
d = df2.ix[:, ['D_1', 'D_3']].values
D_0, D_T = d.T
S_0, S_T = (.1*D_0)**2, np.pi*(.05*D_T)**2
print(np.mean(S_0), np.std(S_0))
print(np.mean(.1*D_T), np.std(.1*D_T))

4.850120899171271 3.2650349587793652
6.232845303867404 2.971909970220621


If I square the diameter I get a mean of 5 square cm's. In the model I initiate with 9 square cm's. Should I follow that? It makes a big difference for the growth factor!

In [25]:
# Now lets say that we start with out 9x9 square and that the expansion is maximally 50% covered
N_total = (fill*S_T - 9)/9
print(np.mean(N_total), np.std(N_total))

1.0804590607826439 2.1068087880222417


In [26]:
p_seeds = []
for B_0, B_T, _ in counts:
    r = (B_T/B_0)**(1/T)
    p = N_total/B_0*(1 - r)/(1 - r**(T+1))
    p_seeds.append(p)
    
print(np.mean(p_seeds), np.std(p_seeds))

0.03294610506340718 0.06457455047638556


This is the number I found, around 3%, to be reasonable for a nice visual result... Is 35% cover for the patch maybe a big on the low side though?... In the calculation I didn't take the biomass of the new ramets into account.